1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
def bin_dec(x):            #from binary to decimal
    result = 0
    iterator = len(x)-1
    for bit in x:
        result += int(bit)*(2**iterator)
        iterator-=1  
        
    return result


def dec_bin(x):           #from decimal to binary
    result = ""
    y = int(x)
    while(y != 0):
        result = str(y%2) + result
        y=y//2 
        
    return result


def dec_hex(x):                                           #from decimal to hexadecimal
    labels = {10:'A',11:'B',12:'C',13:'D',14:'E',15:'F'}
    
    result = ""
    y = int(x)
    while(y != 0):
        reminder = y%16
        if(reminder > 9):
            reminder=labels[reminder]
        result = str(reminder) + result
        y=y//16 

    return result


def hex_dec(x):                                           #from hexadecimal to decimal
    labels = {'A':10,'B':11,'C':12,'D':13,'E':14,'F':15}
    
    result = 0
    iterator = len(x)-1
    for chrt in x:  
        if(chrt.isdigit()):
            chrt=int(chrt)
        else:
            chrt=labels[chrt]      
        result += chrt*(16**iterator)
        iterator-=1
        
    return result
    

    
def transform_rapresentation(x,a,b):
    
    if(a=="bin" and b=="dec"):
        print('Decimal representation of', x, ':', bin_dec(x))
        
    elif(a=="dec" and b=="bin"):        
        print('Binary representation of', x, ':', dec_bin(x))
        
    elif(a=="dec" and b=="hex"):      
        print('Hexadecimal representation of', x, ':', dec_hex(x))
        
    elif(a=="hex" and b=="dec"):
        print('Decimal representation of', x, ':', hex_dec(x))
        
    elif(a=="bin" and b=="hex"):
        print('Hexadecimal representation of', x, ':', dec_hex(bin_dec(x)))
              
    elif(a=="hex" and b=="bin"):
        print('Binary representation of', x, ':', dec_bin(hex_dec(x)))
        


n = "AB38"
ini = "hex"
fin = "bin"
transform_rapresentation(n, ini, fin)

Binary representation of AB38 : 1010101100111000


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
import numpy as np

bin_array = np.random.randint(0, 2, 32)
bin_list = list(map(str, bin_array))
bin_str = "".join(bin_list)

#print(bin_list)
print(bin_str)

00100111001000111100000001010010


In [4]:
def singleprecision(x):
    
    list(x.strip(" "))

    s = int(x[0])
    S=(-1)**s
    #print("\tSign:", s, "=>", S)
    
    e = int(x[1:9], 2)
    #print("\tExponent:", x[1:9], "=>", e)
    
    f = x[9:]
    iterator = -1
    mantissa = 1
    for bit in f: 
        mantissa += int(bit)*(2**iterator)
        #print("iter:", iterator, "bit:", int(bit))
        iterator -= 1  
    #print("\tMantissa:", f, "=>", mantissa)

    bias = 127
    return(S*mantissa*(2**(e-bias)))


#bin_str = "11000000101100000000000000000000"  #"1 00000000 11111111111111111111111"
print("Converting 32bit", bin_str, "into a single precision floating point.")
print("\n\t32bit" ,bin_str, "=>", singleprecision(bin_str), "floating point")

Converting 32bit 00100111001000111100000001010010 into a single precision floating point.

	32bit 00100111001000111100000001010010 => 2.272505117705036e-15 floating point


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
#Overflow

iterator = 1
x,y = 1.0, 1.0

z = y
y *= 2
#print("\nIteration:", iterator,"\nz =", z, "x =", x, "y =", y)
while(x!=y):
    z=x
    x=y
    y*=2
    iterator+=1
    #print("\nIteration:", iterator,"\nz =", z, "x =", x, "y =", y)
    
print("Overflow it's reached after", iterator-1, "iteration with the number limit", z)

Overflow it's reached after 1024 iteration with the number limit 8.98846567431158e+307


In [4]:
#Underflow

iterator = 1
x,y = 1,1

z = x
y /= 2
#print("\nIteration:", iterator,"\nz =", z, "x =", x, "y =", y)
while(x!=y):
    z=x
    x=y
    y/=2
    iterator+=1
    #print("\nIteration:", iterator,"\nz =", z, "x =", x, "y =", y)
    
print("Underflow it's reached after", iterator-1, "iteration with the number limit", z)

Underflow it's reached after 1075 iteration with the number limit 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [21]:
#Machine precision: smallest number ε such that the difference between  
#                   1 and 1 + ε is nonzero ( => distance between 1 and the next  
#                                                                 floating point number )


origin=1
eps=1

iterator=1
#print("\nIteration:", iterator, "epsilon:", eps)
while(origin!=(origin+eps)):
    eps/=2
    iterator+=1
    #print("\nIteration:", iterator, "epsilon:", eps)

print("Machine precision ε = ", 2*eps, "is reached at iteration number", iterator-1)

Machine precision ε =  2.220446049250313e-16 is reached at iteration number 53


In [35]:
import numpy as np

print("Machine precision ε = ", np.finfo(float).eps)
#print(np.finfo(np.float32).eps)                        #Idem but for 32 bit floating points

Machine precision ε =  2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [9]:
from math import sqrt

def equationroots1(a,b,c): 

    discri = (b**2)-4*a*c
    x1 = (-b+sqrt(discri))/(2*a)
    x2 = (-b-sqrt(discri))/(2*a)
    print("Quadratic equation: {}x²+{}x+{}=0".format(a,b,c))
    print("\n=> Solutions [{},{}]".format(x1,x2))  
    
    #print("\n4ac:", 4*a*c)
    #print("\n\nNum +:", (-b+sqrt(discri)))
    #print("Denom +:", (2*a))   

    #print("\n\nNum -:", (-b-sqrt(discri)))
    #print("Denom -:", (2*a)) 

In [10]:
# point (a)

A,B,C = 0.001, 1000, 0.001
equationroots1(A,B,C)

Quadratic equation: 0.001x²+1000x+0.001=0

=> Solutions [-9.999894245993346e-07,-999999.999999]


In [5]:
# point (b)

from math import sqrt

def equationroots2(a,b,c): 

    discri = (b**2)-4*a*c
    x1 = ((-b+sqrt(discri))*(-b-sqrt(discri)))/((2*a)*(-b-sqrt(discri)))
    x2 = ((-b-sqrt(discri))*(-b+sqrt(discri)))/((2*a)*(-b+sqrt(discri)))
    print("Quadratic equation: {}x²+{}x+{}=0".format(a,b,c))
    print("\n=> Solutions [{},{}]".format(x1,x2))  
    
    #print("\n\nNum +:", (-b+sqrt(discri))*(-b-sqrt(discri)))
    #print("Denom +:", (2*a)*(-b-sqrt(discri)))   

    #print("\n\nNum -:", (-b-sqrt(discri))*(-b+sqrt(discri)))
    #print("Denom -:", (2*a)*(-b+sqrt(discri))) 
    
    
    
    
A,B,C = 0.001, 1000, 0.001
equationroots2(A,B,C)

Quadratic equation: 0.001x²+1000x+0.001=0

=> Solutions [-9.999894245993346e-07,-999999.9999990001]


(b) How does it compare with what previously obtained? Why?

$x_2=-999999.9999990001$ has got more significant digist than the previously obtained because we are multipling and dividing the previous $x_2$ by $-b+\sqrt{b^2-4ac}$, that is a number nearby zero. In this way the resuting numerator and denominator own more significant digits. $\newline$

$x_1=-9.999894245993346\cdot10^{-7}$ is the same as the previously obtained because we are multipling and dividing the previous $x_1$ by $-b-\sqrt{b^2-4ac}\backsimeq-2000$, a number that doesn't cause any problems for the floating point precision in this case. Althought we can have some floating point precision's issues at the numerator : we have $-b+\sqrt{b^2-4ac}$ that is a number nearby zero.

In [43]:
# point (c)

from math import sqrt, copysign
from pyfma import fma

def productsDifference(a, b, c, d):    # fma(x. , y. , z.) -> computes (x.*y.) + z. with a single rounding
    dc = d * c
    f = fma(a, b, -dc)                 # f = ab - dc
    return f 

    
def equationroots3(x,y,z): 
    
    print("Quadratic equation: {}x²+{}x+{}=0".format(x, y, z))
    discri = productsDifference(y, y, 4.0*x, z)   #y*y-4*x*z 
    sqrtval = sqrt(abs(discri)) 

    if discri > 0: 
        
        Q = -0.5 * (y + copysign(sqrt(discri), y))
        x1 = Q / x
        x2 = z / Q
        print("\n=> Real and different roots: {", x1, ",", x2, "}" ) 
        
    elif discri == 0: 
        print("\n=> Real coincident roots:", fma(-y, 1/(2*x), 0)) 
        
    else:
        print("\n=> Complex Roots: {", fma(-y, 1/(2*x), 0), "+i", sqrtval, ",", fma(-y, 1/(2*x), 0), 
              "-i", sqrtval, "}") 

        
        
A,B,C = 0.001, 1000, 0.001
equationroots3(A,B,C)      

Quadratic equation: 0.001x²+1000x+0.001=0

=> Real and different roots: { -999999.999999 , -1.000000000001e-06 }


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [48]:
f = lambda y: y*(y-1)

def df_dy(y, delta):
    #print("\nf(y):",f(y))
    #print("\nf(y+delta):",f(y+delta))
    #print("\ny+delta:", y+delta)
    #print("\ny+delta-1:", y+delta-1)
    #print("\nf(y+delta)-f(y):",f(y+delta)-f(y))
    #print("\n(f(y+delta)-f(y))/delta:",(f(y+delta)-f(y))/delta)
    return ((f(y+delta)-f(y))/delta)

In [49]:
# point (a)

from IPython.display import display, Latex

𝛿 = 1e-2
x = 1

display(Latex("Derivative of the function $f(x)=x(x-1)$ at the point $x=1$")) 
display(Latex(" $\diamond$ Computational solution with $\delta = 10^{-2} \Rightarrow %.15f$" 
              % df_dy(x,𝛿)))
display(Latex("$\diamond$ Analytical solution $\Rightarrow 1$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

The two result do not perfectly agree for multiple reasons all related to the finite precision of floating point numbers:
- Computing $f(x+\delta)=f(1+10^{-2})$ we get that $x+\delta-1=1+0.01-1=1.01-1=0.010000000000000009$ and this derives from the fact that we are subtracting two numbers really close to each other. $\newline$
- Computing $f(x+\delta)=(x+\delta)\cdot(x+\delta-1)=1.01\cdot0.010000000000000009=0.010100000000000008$ 
$\newline$This strange truncation of the result can be explained factorizing and recalling that the final sum it's done by the computer using the binary floating point rapresentation: $(1+0.01)\cdot0.010000000000000009=0.00010000000000000009+0.010000000000000009=0.010100000000000008$  $\newline$
- Finally we compute $\frac{f(x+\delta)-f(x)}{\delta}=1.010000000000001$, where the previous last final digit 8 has been truncated to 1: also this result can be explained noticing that the division it's done by the computer using the binary floating point rapresentation.

In [63]:
# point (b)

from IPython.display import display, Latex

l=[]

display(Latex("Derivative of the function $f(x)=x(x-1)$ at the point $x=1$:"))
for i in range(4,15,2):    
    display(Latex("$\diamond$ Computational solution with $\delta = 10^{%i} \Rightarrow %.13f$" 
                  %(i, df_dy(x,10**-i))))
    l.append(df_dy(x,10**-i))
    
    
scale=[]

for j in range(5): 
    rapporto = abs(1-l[j])        # Distance from the true value
    scale.append(rapporto)    

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

The distances from the true value are [9.999999988985486e-05, 9.99917733279787e-07, 3.922528746258536e-09, 8.284037100736441e-08, 8.890058334132256e-05] and they scale approximately by two order of magnitude as delta changes.

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [55]:
from math import sqrt

def RiemannIntegral(a,b,N):
    h = (b-a)/N
    x_k = [a+h*(1/2+i) for i in range(N)]
    y_k = [integrand(j) for j in x_k]
    I = h*sum(y_k)
    return(I)
    
integrand = lambda x: sqrt(1-x**2)


#RiemannIntegral(-1,1,100)

In [57]:
# point (a)

from IPython.display import display, Latex

display(Latex("Computational evaluation of $I=\int_{-1}^{1}\sqrt{1-x^2}dx \Rightarrow I =$ %.6f... with $N = 100$" 
      %RiemannIntegral(-1,1,100)))

<IPython.core.display.Latex object>

My value is 1.57128277... while the true value is $I=\frac{\pi}{2}=1.57079632679...$. $\newline$My value, computed using the midpoint Riemann sum, exeeds the true value because the rectangles used for the integration overtake the shape of the integrationg function.

In [45]:
# point (b) part 1 :  How much can 𝑁 be increased if the computation needs to be run in less than a second?

import timeit as t

loopnum = 1                  #Increase loopnum for an higher accuracy -> the computation time increases a lot!
prev_loop, looptime = 0, 0

for n in range(150000,250000,100):
    prev_loop = looptime
    looptime = t.timeit(lambda: RiemannIntegral(-1,1,n), number = loopnum)
    #print("\n\nN slices:", n, "\tTime needed:", looptime/loopnum)
    #print("\nprevious loop time:", prev_loop)
    
    if(looptime/loopnum > 1):
        break
        
        
#print("\n" ,looptime/loopnum)    
print("\nIn order to run the integration function in less than a second (Result may vary!):")
print("=> 𝑁 can be increased up to", n-100, "at most")
print("=> if 𝑁 =", n-100, "then the computational time needed is %.3fs" %prev_loop)



In order to run the integration function in less than a second (Result may vary!):
=> 𝑁 can be increased up to 159900 at most
=> if 𝑁 = 159900 then the computational time needed is 0.880s


In [4]:
# point (b) part 2 : What is the gain in running it for 1 minute? 

import timeit as t

loopnum = 1                  #Increase loopnum for an higher accuracy -> the computation time increases a lot!
prev_loop, looptime = 0, 0

for n in range(20000000,30000000,10000):
    prev_loop = looptime
    looptime = t.timeit(lambda: RiemannIntegral(-1,1,n), number = loopnum)
    print("\n\nN slices:", n, "\tTime needed:", looptime/loopnum)
    print("\nprevious loop time:", prev_loop)
    
    if(looptime/loopnum > 60):
        print("\n\n\n=> Time needed: %.3fs with integral %.20f and N = %i"
              %(prev_loop, RiemannIntegral(-1,1,n-10000), n-10000))
        break
        



N slices: 20000000 	Time needed: 54.08350389700013

previous loop time: 0


N slices: 20010000 	Time needed: 50.099642154000094

previous loop time: 54.08350389700013


N slices: 20020000 	Time needed: 51.41744563300017

previous loop time: 50.099642154000094


N slices: 20030000 	Time needed: 50.76832504699996

previous loop time: 51.41744563300017


N slices: 20040000 	Time needed: 51.56855190400006

previous loop time: 50.76832504699996


N slices: 20050000 	Time needed: 53.69690818699996

previous loop time: 51.56855190400006


N slices: 20060000 	Time needed: 50.40315527500002

previous loop time: 53.69690818699996


N slices: 20070000 	Time needed: 52.03341534600008

previous loop time: 50.40315527500002


N slices: 20080000 	Time needed: 57.034074575000204

previous loop time: 52.03341534600008


N slices: 20090000 	Time needed: 52.387049298999955

previous loop time: 57.034074575000204


N slices: 20100000 	Time needed: 50.330687872

previous loop time: 52.387049298999955


N